In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd

C:\Users\Hp\anaconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [4]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [5]:
metadata = pd.read_csv(r"C:\Users\Hp\00000000000000 Defence\full_metadata.csv")
metadata.sample(10)

C:\Users\Hp\AppData\Local\Temp\ipykernel_22564\2753292946.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(r"C:\Users\Hp\00000000000000 Defence\full_metadata.csv")


,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,cpu_time,memory,code_size,accuracy
703805,s806019617,p03659,u203927510,1507089452,C,C (GCC 5.4.1),c,Wrong Answer,25.0,1664.0,780,NaN
162464,s626637281,p02263,u240141174,1399567261,C,C,c,Wrong Answer,0.0,552.0,1025,2/10
197057,s933947306,p02279,u147457944,1531224030,C,C,c,Time Limit Exceeded,3990.0,4556.0,2049,6/8
24766,s382612718,p00009,u095123700,1425802529,C,C,c,Compile Error,2740.0,0.0,871,0/1
122201,s491423513,p02241,u421607623,1342560354,C,C,c,Compile Error,0.0,0.0,347,0/6
628622,s511376655,p03262,u061912210,1545179015,C,C (GCC 5.4.1),c,Runtime Error,101.0,128.0,471,NaN
402341,s485023962,p02621,u580231163,1593400482,C,C (GCC 9.2.1),c,Compile Error,NaN,NaN,103,NaN
738512,s836277914,p03909,u932514381,1577851219,C,C (GCC 5.4.1),c,Wrong Answer,1.0,128.0,621,NaN
226429,s258052737,p02388,u367745940,1476797064,C,C,c,Compile Error,0.0,0.0,105,0/4
98476,s061298128,p00734,u064467943,1434105784,C,C,c,Wrong Answer,0.0,612.0,728,0/1


In [6]:
correct_filtered_df = pd.DataFrame()
bug_filtered_df = pd.DataFrame()

In [7]:
def read_code_file(problem_id, submission_id):
    with open("D:\\defence_data\\Project_CodeNet\\data\\{}\\C\\{}.c"
              .format(problem_id, submission_id),encoding="utf8") as f:
        return f.read()

In [8]:
metadata.index.size

754058

In [9]:
indexes = metadata.index
count = 0
for i in indexes[0:]:
    count= count+1
print(count)

754058


In [10]:
count1 = 0
count2 = 0
max_difference = 30
indexes = metadata.index

for i in indexes[0:500]:   # for entire dataset change this to----- for i in indexes[0:-1]:
    problem_id = metadata["problem_id"][i]
    if(metadata["status"][i] == "Runtime Error"):
        bug_submission_id = metadata["submission_id"][i]
        bug_filtered_df = pd.concat([bug_filtered_df, pd.DataFrame(
            {
                "code": read_code_file(problem_id, bug_submission_id),
                "labels": 1
            },
            index=[count1])])
        count1 += 1
    if(metadata["status"][i] == "Accepted"):
        correct_submission_id = metadata["submission_id"][i]
        correct_filtered_df = pd.concat([correct_filtered_df, pd.DataFrame(
            {
                "code": read_code_file(problem_id, correct_submission_id),
                "labels": 0,
            },
            index=[count2])])
        count2 += 1
        
print(count1)
print(count2)

29
233


In [11]:
m_correct = len(correct_filtered_df)
sample_correct = int(m_correct/10)   # train_test_split = 90:10

In [12]:
correct_filtered_df_test = correct_filtered_df[:sample_correct]
correct_filtered_df_train = correct_filtered_df[sample_correct:]

In [13]:
m_bug = len(bug_filtered_df)
sample_bug = int(m_bug/10)

In [14]:
bug_filtered_df_test = bug_filtered_df[:sample_bug]
bug_filtered_df_train = bug_filtered_df[sample_bug:]

In [15]:
final_df_train = pd.concat([correct_filtered_df_train,bug_filtered_df_train],axis=0)
final_df_test = pd.concat([correct_filtered_df_test,bug_filtered_df_test],axis=0)

In [16]:
final_df_train = final_df_train.sample(frac=1).reset_index(drop=True)#USED FOR SHUFFLE
final_df_test = final_df_test.sample(frac=1).reset_index(drop=True)#USED FOR SHUFFLE

In [17]:
final_df_train.to_csv('train.csv',index=False,encoding='utf-8')

In [18]:
final_df_test.to_csv('test.csv',index=False,encoding='utf-8')

In [19]:
(final_df_train['labels']==1).sum()

27

In [20]:
(final_df_test['labels']==1).sum()

2

In [21]:
(final_df_train['labels']==0).sum()

210

In [22]:
(final_df_test['labels']==0).sum()

23

In [23]:
final_df_train

,code,labels
0,#include<stdio.h>\n\nint main(void){\n\n\tint ...,0
1,#include<stdio.h>\n#define N 9\n\nint main(){\...,0
2,#include <stdio.h>\n\nmain()\n{\n\tint i;\n\ti...,0
3,"#include <stdio.h>\nint main(){\n int i, j;...",0
4,"main(){int i,j;for(i=1;i<=9;i++)for(j=1;j<=9;j...",1
...,...,...
232,"#include<stdio.h> \nint main() \n{\n int a,...",0
233,#include<stdio.h>\nint main(void)\n{\n int ...,0
234,"#include<stdio.h>\n\nint main(){\n int i, j...",0
235,"#include<stdio.h>\n\nint main(){\n int x=0,...",0


In [24]:
final_df_test

,code,labels
0,"#include<stdio.h>\nint main(){\n\tint i,j;\n\t...",0
1,j;main(i){for(;j=i<10;i++)for(;j<10;j++)printf...,1
2,#include <stdio.h>\nint main (void)\n{\n\tint ...,0
3,"#include<stdio.h>\n\nint main(){\n int i,j;\n...",0
4,"#include<stdio.h>\nint main(){\n int i=1,j=...",0
5,#include <stdio.h>\n\nint main(void)\n{\n f...,0
6,"#include<stdio.h>\nint main()\n{\n int i,a,...",0
7,"#include<stdio.h>\nint main()\n{\n int i,j;...",0
8,#include<stdio.h>\n\nint main()\n{\n\tfor (int...,0
9,"#include <stdio.h>\nvoid main()\n{\n int j,i...",1


In [25]:
InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1)

In [26]:
def convert_data_to_examples(train, test, code_snippet, label_of_code_snippet): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[code_snippet], 
                                                          text_b = None,
                                                          label = x[label_of_code_snippet]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[code_snippet], 
                                                          text_b = None,
                                                          label = x[label_of_code_snippet]), axis = 1)
  
  return train_InputExamples, validation_InputExamples


In [27]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(final_df_train, 
                                                                           final_df_test, 
                                                                           'code', 
                                                                           'labels')

In [28]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [29]:
code_snippet = 'code'
label_of_code_snippet = 'labels'

In [30]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(final_df_train, final_df_test, code_snippet, label_of_code_snippet)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

C:\Users\Hp\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2198: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [ ]:
model.fit(train_data, epochs=2, validation_data=validation_data,verbose=1,batch_size=16,steps_per_epoch=19)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 1/19 [>.............................] - ETA: 40:39 - loss: 0.6567 - accuracy: 0.6562